In [8]:
%matplotlib qt
from IPython.display import display, HTML
from experiment.experiment_results import ExperimentResult, ResultsTable, get_output_dir

In [9]:
limit = 55

In [10]:
import os

output_dir_ = get_output_dir()

include = ["baseline", "hash"]
exclude = [".stversions", ".ipynb_checkpoints", ".vscode", ".stfolder", "broken", "20iter", "other"]

dirs = [x[0] for x in os.walk(output_dir_)]

# Remove output_dir_ itself
dirs = [x for x in dirs if (x != output_dir_)]
dirs = [x for x in dirs if not any(s in x for s in exclude)]
dirs = [x for x in dirs if any(s in x for s in include)]
folders = [x.split(output_dir_)[1].replace('\\', '') for x in dirs]

#print(folders)

experiments = [ExperimentResult(folder = x, output_dir=output_dir_) for x in folders[:limit]]

for i in range(len(experiments)):
    print(str(i) + ": " + experiments[i].folder)

baseline = experiments[0]


0: baseline_2887iter_1
1: hash_120_eps_0.01_mod_10000_2887iter_1
2: hash_120_eps_0.05_mod_10000_2887iter_1
3: hash_120_eps_0.1_mod_10000_2887iter_1
4: hash_120_eps_0.2_mod_10000_2887iter_1
5: hash_120_eps_0.3_mod_10000_2887iter_1
6: hash_120_eps_0.4_mod_10000_2887iter_1
7: hash_120_eps_0.5_mod_10000_2887iter_1
8: hash_120_eps_0.6_mod_10000_2887iter_1
9: hash_120_eps_0.7_mod_10000_2887iter_1
10: hash_120_eps_0.85_mod_10000_2887iter_1
11: hash_120_eps_0.8_mod_10000_2887iter_1
12: hash_120_eps_0.94_mod_10000_2887iter_1
13: hash_120_eps_0.95_mod_10000_2887iter_1
14: hash_120_eps_0.9_mod_10000_2887iter_1
15: hash_120_eps_1.0_mod_10000_2887iter_1
16: hash_120_eps_1.1_mod_10000_2887iter_1
17: hash_120_eps_1.2_mod_10000_2887iter_1
18: hash_120_eps_1.3_mod_10000_2887iter_1
19: hash_120_eps_1.4_mod_10000_2887iter_1
20: hash_120_eps_1.5_mod_10000_2887iter_1
21: hash_120_eps_1.6_mod_10000_2887iter_1
22: hash_120_eps_1.7_mod_10000_2887iter_1
23: hash_120_eps_1.8_mod_10000_2887iter_1
24: hash_120_ep

**Calculate everything**

In [11]:
for e in experiments[:limit]:
    e.calculate_deviation(reference=baseline)
    e.calculate_averages()

**Data in a table**

In [12]:
rt = ResultsTable()

for e in experiments[:limit]:
    rt.add_to_table(e)


In [13]:
rt.print_table()

,Type,Hash table size,Epsilon,Mod precision,mC/iter,sec/iter,Total mC,Total sec,Deviation average,Deviation maximum,Deviation maximum name,Hits from hash,Writes to hash,Kalman iterations,FLASH memory occupancy (bytes),RAM memory occupancy (bytes)
0,Baseline,0,0,0,5.83872,0.0114893,16.8505,33.1581,0.0000%,0.0000%,Roll,0,0,2887,48640,2296
1,Hash,120,0.01,10000,6.14467,0.0120947,17.7335,34.9054,0.0000%,0.0000%,Roll,0,2887,2887,49792,15752
2,Hash,120,0.05,10000,6.15832,0.0121195,17.7729,34.977,0.0000%,0.0000%,Roll,0,2887,2887,49792,15752
3,Hash,120,0.1,10000,6.17128,0.01215,17.8103,35.065,0.0078%,0.0209%,Kalman Angle Y,1,2886,2887,49792,15752
4,Hash,120,0.2,10000,6.12423,0.011927,17.6745,34.4212,3.4161%,25.9421%,Comp Angle Y,71,2816,2887,49792,15752
5,Hash,120,0.3,10000,5.73474,0.0113163,16.5505,32.6587,9.6413%,66.1240%,Comp Angle Y,236,2651,2887,49792,15752
6,Hash,120,0.4,10000,5.3082,0.0103523,15.3195,29.8768,17.6636%,100.2387%,Comp Angle Y,487,2400,2887,49792,15752
7,Hash,120,0.5,10000,4.80431,0.00946564,13.8652,27.3178,24.0086%,133.3612%,Comp Angle Y,710,2177,2887,49784,15752
8,Hash,120,0.6,10000,4.32533,0.00843966,12.4829,24.3569,32.2725%,185.7147%,Comp Angle Y,971,1916,2887,49792,15752
9,Hash,120,0.7,10000,3.84893,0.00764167,11.108,22.0538,40.3120%,203.1516%,Comp Angle Y,1173,1714,2887,49792,15752


In [8]:
rt.df.to_csv("experiment_results_table.csv")

For all of the below experiments: The hash size is 120. The mod precision is 10 000.

**Kalman iterations** = number of iterations that the Kalman algorithm has actually gone through after the experiments; this is reported by the Kalman results csv

**Power averaged iterations** = number of iterations that were used for the calculation of charge and duration (the **mC/iter**, **sec/iter**, **Total mC**, **Total sec columns**); if it is 1 less than the actual number of iterations, it is because the first iteration has been ignored intentionally since the very first iteration only initializes Kalman and uses different power than the rest of the iterations (which are comparatively in a loop, while the first one is not); this is reported by the power average csv
 Note: the very first iteration can be included in the averaging as an option, and will do so in the future when re-running the averages script at some point

**Deviation iterations** = number of iterations considered for deviation calculations; if this number is smaller in any way, it is wrong; it should be the same number as the baseline number of iterations written also in the table, which is used for reference when calculating deviation; this is reported by a minimum between the iterations of the actual experiment and the reference experiment, which should be the baseline

**Memory sizes**

Read https://mcuoneclipse.com/2013/04/14/text-data-and-bss-code-and-data-size-explained/ for text+data+bss explanation

text = code + constant data that goes into FLASH only

data = initialized data that goes into FLASH, then at startup gets copied to RAM (so counts for both)

bss = uninitialized data that will be given values into RAM

**Plots**

Output plots

In [16]:
experiments[5].experiment_name

'hash_120_eps_0.3_mod_10000_2887iter_1'

In [17]:
experiments[5].plot_output(reference=[baseline])

Power plots

In [18]:
experiments[5].plot_power_curves(reference=baseline)

**Data in text format**

In [ ]:
print("Baseline")
print("---")
baseline.print_output_error(verbose=2)
print("---")
baseline.show_averages(verbose=2)

In [ ]:
print("Hash size: 120, Epsilon: 1, Mod precision: 10000")
print("---")
hash120_eps1.print_output_error(verbose=2)
print("---")
hash120_eps1.show_averages(verbose=2)

In [ ]:
print("Hash size: 120, Epsilon: 0.95, Mod precision: 10000")
print("---")
hash120_eps095.print_output_error(verbose=2)
print("---")
hash120_eps095.show_averages(verbose=2)

In [ ]:
print("Hash size: 120, Epsilon: 0.94, Mod precision: 10000")
print("---")
hash120_eps094.print_output_error(verbose=2)
print("---")
hash120_eps094.show_averages(verbose=2)

**Output Plots**